<a href="https://colab.research.google.com/github/unknown-jun/NLP_study/blob/main/NLP_Book/B05_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as dataset

from torch import Tensor
from torch.autograd import Variable
from torch.nn import Parameter
from torch.utils.data import DataLoader

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

cuda = True if torch.cuda.is_available() else False

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

torch.manual_seed(125)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(125)

In [ ]:
# 데이터셋을 평균과 표준편차로 정규화하고 텐서로 변경함
mnist_transform = transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5,), (1.0,))
])

In [ ]:
# 데이터셋 내려받기 및 전처리 적용
from torchvision.datasets import MNIST
download_root = './MNIST_DATASET'
train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset  = MNIST(download_root, transform=mnist_transform, train=False, download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST_DATASET/MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST_DATASET/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST_DATASET/MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST_DATASET/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST_DATASET/MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST_DATASET/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST_DATASET/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST_DATASET/MNIST/raw



In [ ]:
# 데이터셋 메모리로 가져오기
batch_size = 64
train_loader = DataLoader(dataset = train_dataset,
                          batch_size = batch_size,
                          shuffle=True)
valid_loader = DataLoader(dataset = valid_dataset,
                         batch_size = batch_size,
                         shuffle=True)  # 일반적으로 검증과 테스트 용도의 데이터셋은 섞지 않음
                                        # 예제에선 다양한 학습을 위해 True로 지정
test_loader = DataLoader(dataset = test_dataset,
                         batch_size = batch_size,
                         shuffle=True)

In [ ]:
# 변수값 설정
batch_size = 100
n_iters = 6000
num_epochs = n_iters / (len(train_dataset)/batch_size)
num_epochs = int(num_epochs)

In [ ]:
# GRU 셀 네트워크
class GRUCell(nn.Module):
  def __init__(self, input_size, hidden_size, bias=True):
    super(GRUCell, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.bias = bias 
    self.x2h = nn.Linear(input_size, 3 * hidden_size, bias = bias)
    self.h2h = nn.Linear(input_size, 3 * hidden_size, bias = bias)
    self.reset_parameters()

  def reset_parameters(self): # 파라미터를 초기화
    std = 1.0 / math.sqrt(self.hidden_size)
    for w in self.parameters():
      w.data.uniform_(-std, std)

  def forward(self, x, hidden):
    x = x.view(-1, x.size(1))

    gate_x = self.x2h(x)
    # LSTM 셀에서는 gates를 x2h+h2h로 정의했지만 GRU셀에서는 개별적인 상태를 유지합니다.
    gate_h = self.h2h(hidden)
    
    gate_x = gate_x.squeeze()
    gate_h = gate_h.squeeze()
    
    # 총 3개의 게이트(망각, 입력, 새로운 게이트)를 위해 세 개로 쪼갬
    i_r, i_i, i_n = gate_x.chunk(3,1)
    h_r, h_i, h_n = gate_h.chunk(3,1)

    resetgate = F.sigmoid(i_r + h_r)
    inputgate = F.sigmoid(i_i + h_i)
    # '새로운 게이트'는 탄젠트 활성화 함수가 적용된 게이트
    newgate = F.tahn(i_n + (resetgate * h_n))

    h_y = newgate + inputgate * (hidden - newgate)
    return h_y

In [ ]:
class GRUModel(nn.Module):
  def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True):
    super(GRUModel, self).__init__()
    self.hidden_dim = hidden_dim
    self.layer_dim = layer_dim

    self.gru_cell = GRUCell(input_dim, hidden_dim, layer_dim)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    if torch.cuda.is_available():
      h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
    else:
      h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
      
    outs = []
    hn = h0[0,:,:]

    for seq in range(x.size(1)):
      hn = self.gru_cell(x[:,seq,:], hn)
      outs.append(hn)
      
    out = outs[-1].squeeze()
    out = self.fc(out)

    return out

In [ ]:
# 옵티마이저와 손실 함수 정의
input_dim = 28
hidden_dim = 128
layer_dim = 1
output_dim = 10

model = GRUModel(input_dim, hidden_dim, layer_dim, output_dim)

if torch.cuda.is_available():
  model.cuda()

criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), learning_rate)

In [ ]:
# 모델 학습 및 성능 검증
seq_dim = 28
loss_list = []
iter = 0

for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    if torch.cuda.is_available():
      images = Variable(images.view(-1, seq_dim, input_dim).cuda())
      labels = Variable(labels.cuda())
    else:
      images = Variable(images.view(-1, seq_dim, input_dim))
      labels = Variable(labels)

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)

    if torch.cuda.is_available():
      loss.cuda()
    
    loss.backward()
    optimizer.step()

    loss_list.append(loss.item())
    iter += 1

    if iter % 500 ==0:
      correct = 0
      total = 0

      for images, labels in valid_loader:
        if torch.cuda.is_available():
          images = Variable(images.view(-1, seq_dim, input_dim).cuda())
        else:
          images = Variable(images.view(-1, seq_dim, input_dim))

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)

        if torch.cuda.is_available():
          correct += (predicted.cpu() == labels.cpu()).sum()
        else:
          correct += (predicted == labels).sum()

      accuracy = 100 * correct / total
      print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

RuntimeError: ignored